# TF Learn as TF Estimator
## 6 Steps to Success
1. Find a model that fits your need
2. Write function to import dataset
3. Define the features from the dataset
4. Create an instance of the model
5. Train the model
6. Use the trained model to predict

In [5]:
# 1?
import os
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

# 2
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=False)
# 3                                  
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

n_classes = 10
batch_size = 200
n_steps = 2000
learning_rate = 0.01


def model_fn(features, labels, mode):
    """ define the model function
    """
    espec_op = tf.estimator.EstimatorSpec
    # features is a dict as per Estimator specifications
    x = features['images']
    # define the network
    layer_1 = tf.layers.dense(x, 32)
    layer_2 = tf.layers.dense(layer_1, 32)
    logits = tf.layers.dense(layer_2, n_classes)

    # define predicted classes
    predicted_classes = tf.argmax(logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        espec = espec_op(mode,
                         predictions=predicted_classes
                         )
    else:
        # define loss and optimizer
        entropy_op = tf.nn.sparse_softmax_cross_entropy_with_logits
        loss_op = tf.reduce_mean(entropy_op(logits=logits,
                                            labels=tf.cast(labels,
                                                           dtype=tf.int32)
                                            )
                                 )
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss_op, global_step=tf.train.get_global_step())

        # define accuracy
        accuracy_op = tf.metrics.accuracy(
            labels=labels, predictions=predicted_classes)

        espec = espec_op(mode=mode,
                         predictions=predicted_classes,
                         loss=loss_op,
                         train_op=train_op,
                         eval_metric_ops={'accuracy': accuracy_op}
                         )

    return espec

# 4
# create estimator object
model = tf.estimator.Estimator(model_fn)
# 5
# train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)
model.train(train_input_fn, steps=n_steps)
# 6
# evaluate the model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test},
    y=y_test,
    batch_size=batch_size,
    shuffle=False)
model.evaluate(eval_input_fn)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgsy29b3s', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f220a2f0fd0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpgsy29b3s/model.ckpt.
INFO:tensorflow:loss = 2.4250216, step = 1
INFO:tensorflow:global_step/sec: 98.4118
INFO:tensorflow:loss = 1.50372, step

{'accuracy': 0.9034, 'global_step': 2000, 'loss': 0.33638346}

# TF Slim
## 7 Steps to Success
1. Create the model using slim layers
2. Provide the input to the layers to instantiate the model
3. Use logits and labels to define the loss
4. Get the total loss using convience function ```get_loss_total()```
5. Create optimizer
6. Create a training function using convinience function ```slim.learning.create_train_op()```,```total_loss``` and ```optimizer```
7. Run the training using the convinience function ```slim.learning.train()``` and training function defined in the previous step

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import slim
import tensorflow as tf
import os

tf.reset_default_graph()

n_classes = 10  
n_steps = 1000

# let us get the data
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'), one_hot=True)

X_train = mnist.train.images
X_train = tf.convert_to_tensor(X_train)
Y_train = mnist.train.labels
Y_train = tf.convert_to_tensor(Y_train)


def mlp(x):
    net = slim.fully_connected(x, 32, scope='fc1')
    net = slim.dropout(net, 0.5, scope='dropout1')
    net = slim.fully_connected(net, 32, scope='fc2')
    net = slim.dropout(net, 0.5, scope='dropout2')
    net = slim.fully_connected(net, n_classes, activation_fn=None, scope='fc3')
    return net


# Define the model
logits = mlp(X_train)

# Define the loss functions and get the total loss
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y_train)
total_loss = tf.losses.get_total_loss()

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

train_op = slim.learning.create_train_op(total_loss, optimizer)

# Run the training
final_loss = slim.learning.train(
    train_op,
    logdir='./slim_logs',
    number_of_steps=n_steps,
    log_every_n_steps=100)

print('final loss={}'.format(final_loss))

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path ./slim_logs/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 100: loss = 2.2404 (0.965 sec/step)
INFO:tensorflow:global step 200: loss = 2.1648 (0.817 sec/step)
INFO:tensorflow:global step 300: loss = 2.0825 (0.925 sec/step)
INFO:tensorflow:global step 400: loss = 2.0035 (0.852 sec/step)
INFO:tensorflow:global step 500: loss = 1.9303 (0.808 sec/step)
INFO:tensorflow:global step 600: loss = 1.8626 (0.983 sec/step)


# TFLearn (without the space)
## 7 Steps to Success
1. Create an input layer
2. Pass the object (rock) to create further layers
3. Add the output layer (usualy fully connnected)
4. Create the net using an estimator level such as ```regresssion```.
5. Create a model from the net created in the previous step
6. Train the model with the ```train.fit()``` method
7. Use the trained model to predict or evaluate

In [11]:
import tensorflow as tf
tf.reset_default_graph()

import tflearn
import tflearn.datasets.mnist as mnist
import os

batch_size = 100
n_classes = 10
n_epochs = 10

X_train, Y_train, X_test, Y_test = mnist.load_data(
    data_dir=os.path.join('.', 'mnist'), one_hot=True)

# Build deep neural network
input_layer = tflearn.input_data(shape=[None, 784])
layer1 = tflearn.fully_connected(input_layer,
                                 10,
                                 activation='relu'
                                 )
layer2 = tflearn.fully_connected(layer1,
                                 10,
                                 activation='relu'
                                 )
output = tflearn.fully_connected(layer2,
                                 n_classes,
                                 activation='softmax'
                                 )

net = tflearn.regression(output,
                         optimizer='adam',
                         metric=tflearn.metrics.Accuracy(),
                         loss='categorical_crossentropy'
                         )
model = tflearn.DNN(net)

model.fit(
    X_train,
    Y_train,
    n_epoch=n_epochs,
    batch_size=batch_size,
    show_metric=True,
    run_id='dense_model')

score = model.evaluate(X_test, Y_test)
print('Test accuracy:', score[0])

Training Step: 5499  | total loss: 0.28319 | time: 10.974s
| Adam | epoch: 010 | loss: 0.28319 - acc: 0.9148 -- iter: 54900/55000
Training Step: 5500  | total loss: 0.27340 | time: 10.990s
| Adam | epoch: 010 | loss: 0.27340 - acc: 0.9173 -- iter: 55000/55000
--
Test accuracy: 0.9109


# PrettyTensor
# 8 Steps to Success
1. Get the data
2. Define the hyperparameters and parameters
3. Define the inputs and the outputs
4. Define the model
5. Define the evaluater, optimizer, and trainter functions
6. Create the runner object
7. Within a Tensorflow session, train the model with the ```runner.train_model()``` method
8. Within the same session, evaluate the model with the ```runner.evaluate_model()``` method

In [30]:
import tensorflow as tf
tf.reset_default_graph()

import numpy as np
from __future__ import division

import prettytensor as pt
from prettytensor.tutorial import data_utils
import os
data_utils.WORK_DIRECTORY = os.path.join('.', 'mnist')

# get the data
X_train, Y_train = data_utils.mnist(training=True)
X_test, Y_test = data_utils.mnist(training=False)

# define hyperparameters

batch_size = 100     # number of samples that would
                     # be used to learn the parameters in a batch
n_classes = 10       # number of outputs, i.e. digits 0 to 9
n_epochs = 10        # number of ietrations for learning the parameters
n_batches = int(X_train.shape[0] / batch_size)
n_samples_in_train_batch = 60000 // batch_size
n_samples_in_test_batch = 10000 // batch_size

# define inputs and outputs

X = tf.placeholder(tf.float32, [batch_size, 28, 28, 1])
Y = tf.placeholder(tf.float32, [batch_size, 10])

# define the model

X = pt.wrap(X)

model = (X.
         flatten().
         fully_connected(10).
         softmax_classifier(n_classes, labels=Y)
         )

# define evaluator (metrics), optimizer and training functions

evaluator = model.softmax.evaluate_classifier(Y)
optimizer = tf.train.GradientDescentOptimizer(0.1)
trainer = pt.apply_optimizer(optimizer, losses=[model.loss])

runner = pt.train.Runner()

with tf.Session() as tfs:
    for epoch in range(0, n_epochs):
        # shuffle the training data
        X_train, Y_train = data_utils.permute_data((X_train, Y_train))

        runner.train_model(
            trainer,
            model.loss,
            n_samples_in_train_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_train, Y_train),
            print_every=600
        )

        score = runner.evaluate_model(
            evaluator,
            n_samples_in_test_batch,
            feed_vars=(X, Y),
            feed_data=pt.train.feed_numpy(batch_size, X_test, Y_test)
        )

        print('Accuracy after {} epochs {} \n'.
              format(epoch + 1, score[0]))

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
[1] [2.5092697]
[600] [0.35631377]
Accuracy after 1 epochs 0.9099999666213989 

[601] [0.25389308]
[1200] [0.2711808]
Accuracy after 2 epochs 0.8899999856948853 

[1201] [0.5084967]
[1800] [0.20958649]
Accuracy after 3 epochs 0.8999999761581421 

[1801] [0.23753788]
[2400] [0.34706223]
Accuracy after 4 epochs 0.8899999856948853 

[2401] [0.24079672]
[3000] [0.2273854]
Accuracy after 5 epochs 0.8899999856948853 

[3001] [0.45683563]
[3600] [0.2745242]
Accuracy after 6 epochs 0.8999999761581421 

[3601] [0.35685435]
[4200] [0.44169852]
Accuracy after 7 epochs 0.8999999761581421 

[4201] [0.2994276]
[4800] [0.2605332]
Accuracy after 8 epochs 0.8899999856948853 

[4801] [0.1798949]
[5400] [0.22237508]
Accuracy after 9 epochs 0.8999999761581421 

[5401] [0.15331356]
[6000] [0.315279]
Accuracy after 10 epochs 0.8

# Sonnet
## 8 Steps to Success
1. Create classes for the dataset and network architecture which inherit from ```sonnet.AbstractModuel```. In this example, I have a MNIST class and an MLP class
2. Define the hyperparameters and parameters
3. Define the test and the train datasets from the dataset class defined in the proceeding step.
4. Define the model using the network class defined. As an example, ```model = MLP([20, n_classes])``` in this case creates an MLP network with two layers for 20 and the ```n_classes``` number of neurons each
5. Define the ```y_hat``` placeholders for the train and test set using the model
6. Define the loss placeholders for the train and test sets
7. Define the optimizer using the train loss placeholder
8. Execute the loss function in a Tensorflow session for the desired number of epochs to optimize parameters

In [32]:
import tensorflow as tf
tf.reset_default_graph()

import os
import sonnet as snt

from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.INFO)


class MNIST(snt.AbstractModule):

    def __init__(self, mnist_part, batch_size, name='MNIST'):

        super(MNIST, self).__init__(name=name)

        self._X = tf.constant(mnist_part.images, dtype=tf.float32)
        self._Y = tf.constant(mnist_part.labels, dtype=tf.float32)
        self._batch_size = batch_size
        self._M = mnist_part.num_examples

    def _build(self):
        idx = tf.random_uniform([self._batch_size], 0, self._M, tf.int64)
        X = tf.gather(self._X, idx)
        Y = tf.gather(self._Y, idx)
        return X, Y


class MLP(snt.AbstractModule):
    def __init__(self, output_sizes, name='mlp'):
        super(MLP, self).__init__(name=name)

        self._layers = []

        for output_size in output_sizes:
            self._layers.append(snt.Linear(output_size=output_size))

    def _build(self, X):

        # add the input layer
        model = tf.sigmoid(self._layers[0](X))

        # add hidden layers
        for i in range(1, len(self._layers) - 1):
            model = tf.sigmoid(self._layers[i](model))

        # add output layer
        model = tf.nn.softmax(self._layers[len(self._layers) - 1](model))

        return model


batch_size = 100
n_classes = 10
n_epochs = 10

mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=True
                                  )
train = MNIST(mnist.train, batch_size=batch_size)
test = MNIST(mnist.test, batch_size=batch_size)

X_train, Y_train = train()
X_test, Y_test = test()

model = MLP([20, n_classes])

Y_train_hat = model(X_train)
Y_test_hat = model(X_test)


def loss(Y_hat, Y):
    return -tf.reduce_sum(Y * tf.log(Y_hat))


L_train = loss(Y_train_hat, Y_train)
L_test = loss(Y_test_hat, Y_test)

optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=0.01).minimize(L_train)

with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        loss_val, _ = tfs.run((L_train, optimizer))
        print('Epoch : {} Training Loss : {}'.format(epoch, loss_val))

    loss_val = tfs.run(L_test)
    print('Test loss : {}'.format(loss_val))

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Epoch : 0 Training Loss : 235.41871643066406
Epoch : 1 Training Loss : 228.8345184326172
Epoch : 2 Training Loss : 222.725341796875
Epoch : 3 Training Loss : 223.00149536132812
Epoch : 4 Training Loss : 213.0349884033203
Epoch : 5 Training Loss : 209.04672241210938
Epoch : 6 Training Loss : 209.91319274902344
Epoch : 7 Training Loss : 200.43603515625
Epoch : 8 Training Loss : 203.97579956054688
Epoch : 9 Training Loss : 189.20249938964844
Test loss : 186.88563537597656
